Libraries Used

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error

In [24]:
df= pd.read_csv('C:/Users/Rathore/Downloads/Project Datasets/Movie rating/ratings.dat',sep = '::',names = ['UserID','MovieID','Rating','Timestamp'],engine ='python')
#df= pd.read_csv('C:/Users/Rathore/Downloads/Project Datasets/ml-100k/u.data',sep = '\t',names = ['UserID','MovieID','Rating','Timestamp'],engine ='python')
n_movies = max(df.MovieID)
n_users = max(df.UserID)
df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [25]:
print('No. of users '+ str(n_users)+', No. of Movies'+ str(n_movies))

No. of users 6040, No. of Movies3952


In [26]:
train, test = train_test_split(df,test_size = 0.25)
train.head(10)

,UserID,MovieID,Rating,Timestamp
209398,1279,1875,3,974854171
448906,2766,2716,5,973651175
71171,477,349,4,976219618
550637,3391,521,5,967656876
203746,1253,1288,5,974824567
443079,2731,1147,4,973241907
874354,5278,3744,1,964023443
486379,2994,2723,3,970708400
180278,1128,3844,4,974908339
713283,4277,318,5,965645066


Creating Similarity Metrics

In [27]:
#Training Similarity Metrics:-
train_matrix = np.zeros((n_users,n_movies))
for line in train.itertuples():
    train_matrix[line[1]-1, line[2]-1] = line[3]
#Testing similarity Metrics:-
test_matrix = np.zeros((n_users,n_movies))
for line in test.itertuples():
    test_matrix[line[1]-1, line[2]-1] = line[3]

In [36]:
user_similarity = pairwise_distances(train_matrix, metric = 'cosine')
movie_similarity = pairwise_distances(train_matrix.T, metric = 'cosine')


In [40]:
#np.newaxis so that mean_user_rating has same format as ratings
def predict(ratings,similarity, type = 'user'):
    if type == 'user':
        mean_user_rating  = ratings.mean(axis = 1)
        ratings_diff = (ratings - mean_user_rating[:,np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) /np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'movie':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred    

In [43]:
movie_prediction = predict(train_matrix,movie_similarity, type = 'movie')
user_prediction  = predict(train_matrix,user_similarity, type = 'user')


Root Mean Squared Error (RMSE)

In [46]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return np.sqrt(mean_squared_error(prediction, ground_truth))

In [51]:
print ('User-based Collaborative Filtering  RMSE: ' + str(rmse(user_prediction, test_matrix)))
print ('Item-based Collaborative Filtering  RMSE: ' + str(rmse(movie_prediction, test_matrix)))

User-based Collaborative Filtering  RMSE: 3.25763482583
Item-based Collaborative Filtering  RMSE: 3.53926207119


In [57]:
sparsity=round(1.0-len(df)/float(n_users*n_movies),3)
print ('The sparsity level of MovieLens1M is ' +  str(sparsity*100) + '%')


The sparsity level of MovieLens1M is 95.8%


In [73]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_matrix, k = 25)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print ('User-based CF MSE: ' + str(rmse(X_pred, test_matrix)))

User-based CF MSE: 2.75998872098
